## Lektion 8 - 2025-09-15

### DuckDB

`pip install duckdb`

In [ ]:
import pandas as pd
import duckdb

In [ ]:
data = "C:/Users/jonat/OneDrive/BI25 Python/data/employees.csv"

In [ ]:
schema = {
    "First Name": {"dtype": "string"},
    "Start Date": {"dtype": "datetime64[ns]"},
    "Last Login Time": {"dtype": "string"},
    "Salary": {"dtype": "Int64"},
    "Bonus %": {"dtype": "float"},
    "Senior Management":{"dtype": "boolean"},
    "Team": {"dtype": "string"},
}

rename_map = {
    "First Name": "first_name",
    "Start Date": "start_date",
    "Last Login Time": "last_login_time",
    "Salary": "salary_usd",
    "Bonus %": "bonus_pct",
    "Senior Management": "senior_management",
    "Team": "team",
}

In [ ]:
def apply_schema(df: pd.DataFrame, schema: dict, rename_map: dict) -> pd.DataFrame:
    # datatypes
    for col, datatype in schema.items():
        if col not in df.columns:
            continue

        dtype = datatype.get("dtype", None)
        if dtype and "datetime" in dtype:
            df[col] = pd.to_datetime(df[col], format="%m/%d/%Y", errors="coerce")
        elif dtype:
            df[col] = df[col].astype(dtype, errors="ignore")

    # rename
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

In [ ]:
df = pd.read_csv(data)

In [ ]:
df = apply_schema(df, schema, rename_map)

In [ ]:
df

Med DuckDB går det att skriva SQL queries mot filer, databaser och DataFrames

In [ ]:
duckdb.sql("SELECT 1 + 1 AS simple_addition")

Skriv SQL queries direkt mot ett DataFrame (i det här fallet våran 'df')

In [ ]:
duckdb.sql("SELECT * FROM df WHERE first_name = 'Douglas'")

Samma query men vi lägger till .df() i slutet för att konvertera till ett Pandas DataFrame

In [ ]:
duckdb.sql("SELECT * FROM df WHERE first_name = 'Douglas'").df()

Skriv SQL queryn i en variabel för bättre läsbarhet

In [ ]:
query = """
SELECT
    first_name,
    AVG(salary_usd) AS avg_salary_usd,
    COUNT(first_name) AS count_of_employees
FROM df
WHERE 1 = 1
GROUP BY first_name
ORDER BY avg_salary_usd DESC
LIMIT 5
"""

duckdb.sql(query).df()

Nu raderar vi vårt Pandas DataFrame och provar köra SQL queries direkt mot våran CSV fil 

In [ ]:
# ta bort variabeln df från minnet
del df

In [ ]:
# våran path är kvar
data

SQL query direkt mot CSV filen (utan Pandas)

In [ ]:
duckdb.sql(f"SELECT * FROM '{data}' LIMIT 5")

Vi kan även skriva en SQL query mot CSV-fil som aggregerar snittlön på 'team' kolumnen och spara direkt till en ny CSV-fil

Observera att vi gör detta helt utan att använda DataFrames

In [ ]:
query = f"""
SELECT
    Team,
    AVG(Salary) AS avg_salary_usd
FROM '{data}'
GROUP BY Team
ORDER BY avg_salary_usd DESC
"""

duckdb.sql(query).write_csv("aggregated_employees.csv")

### Koppla upp direkt mot en SQL databas

Det går även att installera olika libraries beroende på vilken databas du använder. T.ex. för PostgreSQL kan man använda `psycopg2` biblioteket

I det här fallet använder vi inga extra libraries, vi använder DuckDB funktionalitet för att koppla upp oss mot lokala PostgreSQL databasen

In [ ]:
con = duckdb.connect("demo.duckdb")

con.execute("INSTALL postgres;")
con.execute("LOAD postgres;")

Observera att nedan är connection detaljer för min lokala databas, den kommer inte att fungera att köra om man inte har en identisk uppsättning lokalt

In [ ]:
pg_conn = "dbname=BI25DB user=postgres password=Linux4Ever host=localhost port=5432"

Hämta alla rader från raw_employees tabellen

In [ ]:
df = con.execute(f"""
    SELECT * 
    FROM postgres_scan('{pg_conn}', 'public', 'raw_employees')
""").df()

In [ ]:
df.head()

Eftersom att vi körde .df() när vi hämtade data från Postgres så skapades det en Pandas DataFrame

In [ ]:
type(df)

Vi kan som vanligt köra SQL queries mot Pandas DataFramet

In [ ]:
duckdb.sql("SELECT * FROM df WHERE employee_id LIKE '%02'")

Skapa ett nytt Pandas DataFrame med enbart DuckDB kod

In [ ]:
new_df = duckdb.sql("SELECT * FROM df WHERE employee_id LIKE '%02'").df()

Utföra våra vanliga Pandas-operationer på DataFramet som DuckDB skapade

In [ ]:
new_df["name"].value_counts()